In [1]:
%load_ext sql
%config SqlMagic.feedback = False

In [2]:
from os import environ

# Obtain local development environment Postgres connection settings
if environ.get("UDACITY_DATA_ENGINEER_POSTGRES_INSTANCE"):
    host = environ.get("UDACITY_DATA_ENGINEER_POSTGRES_INSTANCE")
    user = environ.get("UDACITY_DATA_ENGINEER_POSTGRES_USER")
    password = environ.get("UDACITY_DATA_ENGINEER_POSTGRES_PASSWORD")

conn_string = "postgresql://{}:{}@{}/sparkifydb".format(user, password, host)

In [18]:
%sql $conn_string SELECT * FROM songplays LIMIT 5;

songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
1,2018-11-21 19:03:52.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
2,2018-11-21 19:09:10.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
3,2018-11-21 19:12:06.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
4,2018-11-21 19:15:41.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
5,2018-11-21 19:23:40.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""


In [4]:
%sql $conn_string SELECT * FROM users LIMIT 5;

user_id,first_name,last_name,gender,level
15,Lily,Koch,F,paid
26,Ryan,Smith,M,free
50,Ava,Robinson,F,free
97,Kate,Harrell,F,paid
101,Jayden,Fox,M,free


In [5]:
%sql $conn_string SELECT * FROM songs LIMIT 5;

song_id,title,artist_id,year,duration
SOUDSGM12AC9618304,Insatiable (Instrumental Version),ARNTLGG11E2835DDB9,0,266.39627
SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
SOXVLOJ12AB0189215,Amor De Cabaret,ARKRRTF1187B9984DA,0,177.47546
SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.9318
SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608


In [6]:
%sql $conn_string SELECT * FROM artists LIMIT 5;

artist_id,name,location,latitude,longitude
ARNTLGG11E2835DDB9,Clp,,nan,nan
AR7G5I41187FB4CE6C,Adam Ant,"London, England",nan,nan
ARKRRTF1187B9984DA,Sonora Santanera,,nan,nan
ARD7TVE1187B99BFB1,Casual,California - LA,nan,nan
ARXR32B1187FB57099,Gob,,nan,nan


In [7]:
%sql $conn_string SELECT * FROM time LIMIT 5;

start_time,hour,day,week,month,year,weekday
2018-11-22 00:03:52.796000,0,22,47,11,2018,3
2018-11-22 00:09:10.796000,0,22,47,11,2018,3
2018-11-22 00:12:06.796000,0,22,47,11,2018,3
2018-11-22 00:15:41.796000,0,22,47,11,2018,3
2018-11-22 00:23:40.796000,0,22,47,11,2018,3


## REMEMBER: Restart this notebook to close connection to `sparkifydb`
Each time you run the cells above, remember to restart this notebook to close the connection to your database. Otherwise, you won't be able to run your code in `create_tables.py`, `etl.py`, or `etl.ipynb` files since you can't make multiple connections to the same database (in this case, sparkifydb).

## Troubleshooting song_id SOZCTXZ12AB0182364 Missing From JSON Log Files

In [41]:
!grep -ir SOZCTXZ12AB0182364 ./data

./data/song_data/A/A/C/TRAACCG128F92E8A55.json:{"num_songs": 1, "artist_id": "AR5KOSW1187FB35FF4", "artist_latitude": 49.80388, "artist_longitude": 15.47491, "artist_location": "Dubai UAE", "artist_name": "Elena", "song_id": "SOZCTXZ12AB0182364", "title": "Setanta matins", "duration": 269.58322, "year": 0}


As shown in the result above, there is not a log record for "SOZCTXZ12AB0182364". At some point this JSON log file was removed. Therefore cannot me requirement of one record returning with non-NULL `song_id` and `artist_id` values. See https://knowledge.udacity.com/questions/48698 for a discussion of the issue.

In [17]:
%sql $conn_string SELECT songs.song_id, artists.artist_id FROM songs INNER JOIN artists ON songs.artist_id = artists.artist_id WHERE songs.title = 'Face the Ashes' AND artists.name = 'Gob' AND songs.duration = '209.60608'

song_id,artist_id
SOFSOCN12A8C143F5D,ARXR32B1187FB57099


In [12]:
%sql $conn_string SELECT * from artists WHERE artist_id = 'ARXR32B1187FB57099'

artist_id,name,location,latitude,longitude
ARXR32B1187FB57099,Gob,,nan,nan


In [23]:
%sql $conn_string SELECT * FROM songplays WHERE song_id IS NULL LIMIT 5;

songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
1,2018-11-21 19:03:52.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
2,2018-11-21 19:09:10.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
3,2018-11-21 19:12:06.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
4,2018-11-21 19:15:41.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
5,2018-11-21 19:23:40.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""


In [39]:
%sql $conn_string SELECT song_id, COUNT(*) FROM songplays GROUP BY song_id;

song_id,count
None,6820


In [29]:
%sql $conn_string SELECT * FROM songplays WHERE session_id = 818 

songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
1,2018-11-21 19:03:52.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
2,2018-11-21 19:09:10.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
3,2018-11-21 19:12:06.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
4,2018-11-21 19:15:41.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
5,2018-11-21 19:23:40.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
6,2018-11-21 19:28:26.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
7,2018-11-21 19:32:25.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
8,2018-11-21 19:38:50.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
9,2018-11-21 19:41:49.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""
10,2018-11-21 19:45:59.796000,15,paid,None,None,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"""


In [40]:
%sql $conn_string SELECT * FROM songplays WHERE song_id = 'SOZCTXZ12AB0182364'

songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent


In [34]:
%sql $conn_string SELECT * FROM songs WHERE song_id = 'SOZCTXZ12AB0182364'

song_id,title,artist_id,year,duration
SOZCTXZ12AB0182364,Setanta matins,AR5KOSW1187FB35FF4,0,269.58322
